<a href="https://colab.research.google.com/github/TAlkam/Identifying-Risk-Factors-for-In-Hospital-Mortality-in-Alzheimer-s-Disease/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 📘 Notebook Outline

This notebook presents a dual-method analysis of in-hospital mortality in Alzheimer’s disease (AD) patients using the 2017 NIS dataset. It includes data preprocessing, logistic regression, XGBoost modeling, and SHAP explainability.

### 1. **Setup and Imports**

* Install and import required Python packages.

### 2. **Data Loading**

* Load `.dta` file using `pyreadstat`.
* Preview dataset dimensions and contents.

### 3. **Diagnosis Code Extraction**

* Extract and flatten `I10_DX1–I10_DX40` diagnosis columns.
* Exclude Alzheimer’s/dementia and Z-codes (optional).
* Merge with ICD-10 descriptions.

### 4. **Top Diagnosis Frequency Analysis**

* Count and display top 30 most frequent diagnoses.

### 5. **Feature Engineering**

* Define binary, categorical, and continuous variables.
* Create target variable (`died`).
* Preprocess features for modeling.

### 6. **Modeling: Logistic Regression & XGBoost**

* Apply stratified train-validation-test split.
* Train logistic regression (with survey weights).
* Train XGBoost classifier with early stopping.

### 7. **Model Evaluation**

* Report AUROC, AUPRC, Brier Score, and LogLoss.
* Cross-validation performance metrics.

### 8. **Model Explainability (SHAP)**

* Generate SHAP summary and bar plots for XGBoost.
* Identify key features contributing to mortality prediction.

### 9. **Sensitivity Analysis**

* Re-train models after excluding `dnr` and `pall` variables.
* Recompute SHAP and evaluate performance change.

### 10. **Export and Save**

* Save final model, SHAP plots, and top features.


In [ ]:
# 📦 Install required packages
!pip install pyreadstat pandas openpyxl --quiet

# 📚 Import libraries
import pandas as pd
import pyreadstat

# 📂 Load your .dta file
df, meta = pyreadstat.read_dta('/content/NIS_2017_Core data age_over_60_alz_only-DIED.dta')

# ✅ Step 1: Extract all I10_DX* diagnosis code columns
dx_cols = [col for col in df.columns if col.lower().startswith('i10_dx')]
print(f"Diagnosis columns: {dx_cols[:5]} ... total: {len(dx_cols)}")

# ✅ Step 2: Stack all diagnosis codes into a single Series
all_dx = df[dx_cols].values.ravel()  # Flatten all diagnosis codes
all_dx = pd.Series(all_dx).dropna().astype(str).str.upper().str.strip()

# ❌ DO NOT exclude dementia, Z-codes, or anything else — include all ICD-10 codes
# ✅ Step 3: Count frequencies of ALL ICD-10 diagnosis codes
dx_counts = all_dx.value_counts().reset_index()
dx_counts.columns = ['icd10_code', 'count']
dx_counts['percent'] = dx_counts['count'] / dx_counts['count'].sum() * 100

# 📂 Step 4: Load ICD‑10 code descriptions (make sure filename is correct)
icd_lookup = pd.read_excel('/content/section111validicd10-jan2025_0.xlsx')

# 🧹 Step 5: Clean and standardize
icd_lookup.columns = icd_lookup.columns.str.lower().str.strip()
icd_lookup = icd_lookup.rename(columns={
    'code': 'icd10_code',
    'long description (valid icd-10 fy2025)': 'description'
})
icd_lookup['icd10_code'] = icd_lookup['icd10_code'].str.upper().str.strip()
dx_counts['icd10_code'] = dx_counts['icd10_code'].str.upper().str.strip()

# 🔗 Step 6: Merge frequencies with ICD-10 descriptions
merged = dx_counts.merge(icd_lookup[['icd10_code', 'description']], on='icd10_code', how='left')

# 🧽 Step 7: Optionally drop missing descriptions, or keep them for manual review
merged = merged.dropna(subset=['description'])  # or comment this out to keep all rows

# 📊 Step 8: Display top 30 most frequent diagnosis codes (now includes Z-codes)
merged_top30 = merged[['icd10_code', 'description', 'count', 'percent']].head(30)
display(merged_top30)


In [ ]:
# 📦 Install required packages
!pip install pyreadstat pandas openpyxl --quiet

# 📚 Import libraries
import pandas as pd
import pyreadstat

# 📂 Load your .dta file
df, meta = pyreadstat.read_dta('/content/NIS_2017_Core data age_over_60_alz_only-DIED.dta')  # Update path if needed

# ✅ Step 1: Extract all I10_DX* columns (diagnosis codes)
dx_cols = [col for col in df.columns if col.lower().startswith('i10_dx')]

# ✅ Step 2: Stack all diagnosis codes into a single Series
all_dx = df[dx_cols].values.ravel()  # Flatten 2D array
all_dx = pd.Series(all_dx).dropna().str.upper().str.strip()

# ✅ Step 3: Exclude dementia and Alzheimer's codes (F01*, F02*, F03*, G30*, Z-codes)
exclude_prefixes = ['F01', 'F02', 'F03', 'G30']
all_dx_filtered = all_dx[~all_dx.str.startswith(tuple(exclude_prefixes))]
all_dx_filtered = all_dx_filtered[~all_dx_filtered.str.startswith('Z')]

# ✅ Step 4: Count frequencies of remaining ICD-10 codes
dx_counts = all_dx_filtered.value_counts().reset_index()
dx_counts.columns = ['icd10_code', 'count']
dx_counts['percent'] = dx_counts['count'] / dx_counts['count'].sum() * 100

# 📂 Step 5: Load ICD‑10 code descriptions (update filename if needed)
icd_lookup = pd.read_excel('/content/section111validicd10-jan2025_0.xlsx')

# 🧹 Clean and standardize
icd_lookup.columns = icd_lookup.columns.str.lower().str.strip()
icd_lookup = icd_lookup.rename(columns={
    'code': 'icd10_code',
    'long description (valid icd-10 fy2025)': 'description'
})
icd_lookup['icd10_code'] = icd_lookup['icd10_code'].str.upper().str.strip()
dx_counts['icd10_code'] = dx_counts['icd10_code'].str.upper().str.strip()

# 🔗 Step 6: Merge diagnosis counts with descriptions
merged = dx_counts.merge(icd_lookup[['icd10_code', 'description']], on='icd10_code', how='left')

# ✅ Step 7: Drop NaN descriptions (this removes missing rows)
merged = merged.dropna(subset=['description'])

# 📊 Step 8: Display top 30 diagnoses
merged_top30 = merged[['icd10_code', 'description', 'count', 'percent']].head(30)
merged_top30


In [ ]:
import matplotlib.pyplot as plt
import textwrap

# Drop rows with missing descriptions
filtered = merged.dropna(subset=['description'])

# Remove Z codes (non-disease administrative/status codes)
filtered = filtered[~filtered['icd10_code'].str.startswith('Z', na=False)]

# Take top 20
top_dx = filtered.head(30).copy()
top_dx['description'] = top_dx['description'].astype(str)

# Wrap long descriptions for better visibility
top_dx['wrapped_desc'] = top_dx['description'].apply(lambda x: '\n'.join(textwrap.wrap(x, width=50)))

# Plot with larger fonts
plt.figure(figsize=(14, 20))
plt.barh(top_dx['wrapped_desc'], top_dx['count'])
plt.xlabel('Frequency', fontsize=14)
plt.title('Top 30 Diagnoses Among AD Patients Died In Hospital', fontsize=20)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


In [ ]:
!pip uninstall -y shap numpy scikit-learn xgboost
!pip install numpy==1.26.4 shap==0.44.1 scikit-learn==1.4.2 xgboost==2.1.1 matplotlib==3.9.2


In [ ]:
# ================================================================
# MACHINE LEARNING PIPELINE (COLAB‑FRIENDLY, FULLY FIXED)
# ================================================================

import os, warnings, numpy as np, pandas as pd
warnings.filterwarnings("ignore")

from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss, log_loss
from xgboost import XGBClassifier
import shap

# ==== 0) I/O SETUP ======================================================
USE_DRIVE = True
PATH_DTA = "/content/NIS_2017_Core data age_over_60 new labels_alz-only for analysis.dta"

if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
else:
    from google.colab import files
    up = files.upload()
    PATH_DTA = list(up.keys())[0]

# ==== 1) SAFE LOADER =====================================================
def read_stata_robust(path):
    """Read .dta file safely, fallback to latin‑1 if needed."""
    try:
        return pd.read_stata(path, convert_categoricals=False, encoding="latin-1")
    except TypeError:
        return pd.read_stata(path, convert_categoricals=False)

df = read_stata_robust(PATH_DTA)
df.columns = df.columns.str.lower()

# De‑duplicate columns (Stata exports often duplicate 'female', 'race')
if pd.Index(df.columns).duplicated().any():
    dup_names = pd.Series(df.columns)[pd.Index(df.columns).duplicated(keep=False)].unique().tolist()
    print("⚠️ Duplicate column names detected; keeping the first occurrence for:", dup_names)
    df = df.loc[:, ~pd.Index(df.columns).duplicated()].copy()

def as_series(x):
    import pandas as pd
    return x.iloc[:, 0] if isinstance(x, pd.DataFrame) else x

# ==== 2) VARIABLE DECLARATIONS ==========================================
TARGET = "died"
GROUPS = "hosp_nis"
WEIGHT = "discwt"

expected_binary = [
    "female","elective","aweekend","sepsis","arf","aki","uti","asp","malnut",
    "dysph","pressulc","chf","cad","afib","cva","anemia","hypoth","dnr","pall"
]
expected_cats = ["race","zipinc_qrtl","tran_in","hosp_division"]
expected_cont = ["age"]

# ==== 3) BASIC CLEANING =================================================
all_feature_names = expected_cont + expected_binary + expected_cats
avail = [c for c in all_feature_names if c in df.columns]
missing = sorted(set(all_feature_names) - set(avail))
if missing:
    print("⚠️ Missing columns (proceeding without them):", missing)

keep = [c for c in [TARGET, GROUPS, WEIGHT] if c in df.columns] + avail
df = df[keep].copy()

# Safe numeric conversion
for c in expected_cont + expected_binary + expected_cats:
    if c in df:
        s = as_series(df[c])
        df[c] = pd.to_numeric(s, errors="coerce")

# ==== 4) TARGET FIX =====================================================
y_raw = pd.to_numeric(df[TARGET], errors="coerce")
y = np.where(y_raw >= 1, 1, 0).astype(int)
print("✅ Target cleaned. Unique values after recoding:", np.unique(y))

# ==== 5) GROUPS & WEIGHTS ===============================================
groups = pd.to_numeric(df.get(GROUPS, pd.Series(np.arange(len(df)), index=df.index)),
                       errors="coerce").fillna(-1).astype(int).values

w = pd.to_numeric(df.get(WEIGHT, pd.Series(1, index=df.index)), errors="coerce").fillna(1.0)
w = w.replace([np.inf, -np.inf], np.nan).fillna(1e-6)
w = w.clip(lower=1e-6, upper=w.quantile(0.999))  # cap extreme outliers
w = w.astype(float).values
print(f"✅ Final cleaned weights: min={w.min():.4e}, max={w.max():.4e}")

# ==== 6) FEATURES =======================================================
X = df[avail].copy()
for c in X.columns:
    if c in expected_cont:
        X[c] = X[c].astype(float).fillna(X[c].median())
    elif c in expected_binary:
        X[c] = X[c].fillna(0).astype(int)
    elif c in expected_cats:
        X[c] = X[c].fillna(X[c].mode(dropna=True).iloc[0] if X[c].notna().any() else 0).astype(int)
    else:
        X[c] = X[c].fillna(0)

num_feats = [c for c in expected_cont if c in X] + [c for c in expected_binary if c in X]
cat_feats = [c for c in expected_cats if c in X]

# ==== 7) PREPROCESSOR ===================================================
pre = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_feats),
        ("cat", "passthrough", cat_feats)
    ],
    remainder="drop"
)

# ==== 8) MODEL TRAINER (SAFE WEIGHTS) ==================================
def fit_with_weights(pipe, X_tr, y_tr, w_tr):
    w_tr = pd.Series(w_tr).replace([np.inf, -np.inf], np.nan).fillna(1e-6).clip(lower=1e-6, upper=100)
    try:
        pipe.fit(X_tr, y_tr, clf__sample_weight=w_tr)
    except TypeError:
        pipe.fit(X_tr, y_tr)
    return pipe

# ==== 9) MODELS =========================================================
logit = Pipeline([
    ("pre", pre),
    ("clf", LogisticRegression(solver="lbfgs", max_iter=1000))
])

xgb = Pipeline([
    ("pre", pre),
    ("clf", XGBClassifier(
        n_estimators=600, learning_rate=0.05, max_depth=4,
        subsample=0.9, colsample_bytree=0.9,
        reg_lambda=1.0, reg_alpha=0.0,
        random_state=42, tree_method="hist", n_jobs=-1,
        eval_metric="logloss"
    ))
])

MODELS = {"Logistic": logit, "XGBoost": xgb}

# ==== 10) METRICS =======================================================
def weighted_metrics(y_true, y_prob, sample_weight):
    y_true = np.asarray(y_true); y_prob = np.asarray(y_prob); sw = np.asarray(sample_weight)
    eps = 1e-9
    return dict(
        AUROC = roc_auc_score(y_true, y_prob, sample_weight=sw),
        AUPRC = average_precision_score(y_true, y_prob, sample_weight=sw),
        Brier = brier_score_loss(y_true, np.clip(y_prob, eps, 1-eps), sample_weight=sw),
        LogLoss = log_loss(y_true, np.clip(y_prob, eps, 1-eps), sample_weight=sw)
    )

# ==== 11) GROUPED CV ====================================================
N_SPLITS = 5
gkf = GroupKFold(n_splits=N_SPLITS)

summaries, oof_store = [], {}

for name, pipe in MODELS.items():
    print(f"\n▶ Running {name} ...")
    y_all=[]; p_all=[]; w_all=[]; fold_id=[]; idx_all=[]
    for fold, (tr, te) in enumerate(gkf.split(X, y, groups=groups), start=1):
        X_tr, X_te = X.iloc[tr], X.iloc[te]
        y_tr, y_te = y[tr], y[te]
        w_tr, w_te = w[tr], w[te]
        fitted = fit_with_weights(pipe, X_tr, y_tr, w_tr)
        p_te = fitted.predict_proba(X_te)[:,1]
        y_all.append(y_te); p_all.append(p_te); w_all.append(w_te)
        fold_id.append(np.full_like(y_te, fold)); idx_all.append(te)

    y_all = np.concatenate(y_all); p_all = np.concatenate(p_all)
    w_all = np.concatenate(w_all); fold_id = np.concatenate(fold_id); idx_all = np.concatenate(idx_all)
    met = weighted_metrics(y_all, p_all, w_all)
    summaries.append({"Model": name, **{k: float(v) for k,v in met.items()}})
    oof_store[name] = pd.DataFrame({"idx": idx_all, "fold": fold_id, "y": y_all, "p": p_all, "w": w_all}).sort_values("idx")

summary_df = pd.DataFrame(summaries).sort_values("AUROC", ascending=False)
print("\n=== Cross‑validated, weighted metrics (OOF) ===")
print(summary_df.to_string(index=False, formatters={k: "{:.4f}".format for k in ["AUROC","AUPRC","Brier","LogLoss"]}))

for k, df_oof in oof_store.items():
    outp = f"/content/{k}_oof_predictions.csv"
    df_oof.to_csv(outp, index=False)
    print("Saved:", outp)

# ==== 12) INTERPRETATION ================================================
print("\n=== Logistic regression coefficients (full fit) ===")
logit_full = fit_with_weights(MODELS["Logistic"], X, y, w)
feat_names = num_feats + cat_feats
coef = logit_full.named_steps["clf"].coef_.ravel()
logit_imp = pd.DataFrame({"feature": feat_names, "coef": coef, "abs_coef": np.abs(coef)}).sort_values("abs_coef", ascending=False)
print(logit_imp.head(25).to_string(index=False))

if "XGBoost" in MODELS:
    print("\n=== XGBoost feature importance (gain) ===")
    xgb_full = fit_with_weights(MODELS["XGBoost"], X, y, w)
    xgb_feat_names = feat_names
    booster = xgb_full.named_steps["clf"]
    gain_scores = booster.get_booster().get_score(importance_type="gain")
    rows = [{"feature": fname, "gain": gain_scores.get(f"f{i}", 0.0)} for i, fname in enumerate(xgb_feat_names)]
    xgb_imp = pd.DataFrame(rows).sort_values("gain", ascending=False)
    print(xgb_imp.head(25).to_string(index=False))

    # Get transformed feature matrix and reattach column names
X_mat = xgb_full.named_steps["pre"].transform(X)

# Combine numerical + categorical feature names after preprocessing
transformed_feature_names = (
    xgb_full.named_steps["pre"]
    .transformers_[0][2] +  # StandardScaler features
    xgb_full.named_steps["pre"]
    .transformers_[1][2]    # Categorical passthrough
)

# Turn it into a DataFrame with feature names
X_df = pd.DataFrame(X_mat, columns=transformed_feature_names)


import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Re-transform your X into DataFrame with numeric values
X_mat = xgb_full.named_steps["pre"].transform(X)

# Get column names from the preprocessor
num_feats = xgb_full.named_steps["pre"].transformers_[0][2]  # numerical
cat_feats = xgb_full.named_steps["pre"].transformers_[1][2]  # categorical
all_feats = num_feats + cat_feats

# Ensure numeric conversion from string, remove brackets if needed
def sanitize(x):
    if isinstance(x, str):
        x = x.replace('[', '').replace(']', '')
    try:
        return float(x)
    except:
        return np.nan

X_mat_clean = np.vectorize(sanitize)(X_mat)  # Converts all to float
X_df = pd.DataFrame(X_mat_clean, columns=all_feats)

# Sample subset for SHAP
n_sample = min(5000, len(X_df))
idx = np.random.default_rng(42).choice(len(X_df), size=n_sample, replace=False)
X_sample = X_df.iloc[idx].copy()

# Fit SHAP explainer on the XGBoost model
explainer = shap.Explainer(xgb_full.named_steps["clf"])
shap_values = explainer(X_sample)

# Plot SHAP summary
# === Custom Labeled SHAP Bar Plot ===
import matplotlib.pyplot as plt

# Compute mean absolute SHAP values for each feature
mean_shap_vals = np.abs(shap_values.values).mean(axis=0)
shap_df = pd.DataFrame({
    "feature": X_sample.columns,
    "mean_abs_shap": mean_shap_vals
}).sort_values("mean_abs_shap", ascending=False)

# Define your custom readable labels for the top features

label_dict = {
    "female": "Sex",
    "elective": "Elective Admission",
    "aweekend": "Weekend Admission",
    "sepsis": "Sepsis",
    "arf": "Acute Respiratory Failure",
    "aki": "Acute Kidney Injury",
    'age': 'Age',
    "uti": "Urinary Tract Infection",
    "asp": "Aspiration Pneumonia",
    "malnut": "Malnutrition",
    "dysph": "Dysphagia",
    "pressulc": "Pressure Ulcer",
    "chf": "Congestive Heart Failure",
    "cad": "Coronary Artery Disease",
    "afib": "Atrial Fibrillation",
    "cva": "Stroke (Cerebrovascular Accident)",
    "anemia": "Anemia",
    "hypoth": "Hypothyroidism",
    "dnr": "Do Not Resuscitate",
    "pall": "Palliative Care",
    "race": "Race/Ethnicity",
    "zipinc_qrtl": "Income Quartile (by ZIP Code)",
    "tran_in": "Transferred In",
    "hosp_division": "Hospital Region"
}


# Map labels, keeping original names if no match
shap_df['label'] = shap_df['feature'].map(label_dict).fillna(shap_df['feature'])

# Plot
plt.figure(figsize=(10, 8))
plt.barh(shap_df['label'].iloc[::-1], shap_df['mean_abs_shap'].iloc[::-1], color='dodgerblue')
plt.xlabel("mean(|SHAP value|) (average impact on model output magnitude)")
plt.title("SHAP Feature Importance (Top Predictors of Mortality)")
plt.tight_layout()
plt.show()

# ==== 13) THRESHOLD METRICS =============================================
def best_threshold_by_youden(y_true, y_prob, sample_weight=None):
    from sklearn.metrics import roc_curve
    fpr, tpr, thr = roc_curve(y_true, y_prob, sample_weight=sample_weight)
    j = tpr - fpr
    i = np.argmax(j)
    return float(thr[i]), float(tpr[i]), float(fpr[i])

for name, df_oof in oof_store.items():
    thr, tpr, fpr = best_threshold_by_youden(df_oof["y"], df_oof["p"], df_oof["w"])
    print(f"{name}: Youden‑optimal threshold ~ {thr:.3f} (TPR={tpr:.3f}, FPR={fpr:.3f})")

summary_df.to_csv("/content/ML_cv_summary.csv", index=False)
print("\n✅ Saved summary: /content/ML_cv_summary.csv")


In [ ]:
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Define full feature name mapping
feature_name_map = {
    'pall': 'Palliative Care',
    'zipinc_qrtl': 'ZIP Income Quartile',
    'arf': 'Acute Respiratory Failure',
    'aki': 'Acute Kidney Injury',
    'age': 'Age',
    'female': 'Sex',
    'race': 'Race',
    'sepsis': 'Sepsis',
    'chf': 'Congestive Heart Failure',
    'asp': 'Aspiration Pneumonia',
    'aweekend': 'Weekend Admission',
    'malnut': 'Malnutrition',
    'uti': 'Urinary Tract Infection',
    'tran_in': 'Transferred In',
    'dysph': 'Dysphagia',
    'cad': 'Coronary Artery Disease',
    'dnr': 'DNR Status',
    'hypoth': 'Hypothyroidism',
    'pressulc': 'Pressure Ulcer',
    'afib': 'Atrial Fibrillation'
}

# Step 2: Transform input and sample
X_mat = xgb_full.named_steps['pre'].transform(X)
X_mat = pd.DataFrame(X_mat, columns=xgb_feat_names)
X_sample = X_mat.sample(n=1000, random_state=42)

# Step 3: Compute SHAP values
explainer = shap.Explainer(xgb_full.named_steps['clf'])
shap_values = explainer(X_sample)

# Step 4: Filter top features
top_features = list(feature_name_map.keys())
top_indices = [i for i, f in enumerate(shap_values.feature_names) if f in top_features]
shap_values_filtered = shap_values[:, top_indices]

# Step 5: Rename features inside SHAP object
# This replaces the feature names in-place (works on the view, not original)
for i, idx in enumerate(top_indices):
    shap_values_filtered.feature_names[i] = feature_name_map[shap_values.feature_names[idx]]

# Step 6: Plot beeswarm with full names
shap.plots.beeswarm(shap_values_filtered, max_display=20)


**SHAP Mean Absolute Values**

In [ ]:
import pandas as pd
shap_df = pd.DataFrame({
    "Feature": shap_values.feature_names,
    "SHAP_Mean_Abs": np.abs(shap_values.values).mean(axis=0)
}).sort_values("SHAP_Mean_Abs", ascending=False)
display(shap_df)

In [ ]:
# Create a reduced feature set excluding end-of-life care variables
exclude_vars = ['dnr', 'pall']
X_sensitivity = X.drop(columns=exclude_vars)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Fit logistic regression without DNR and palliative care
lr_sens = LogisticRegression(max_iter=1000)
lr_sens.fit(X_sensitivity, y)

# Predict and compute AUROC
y_pred_lr_sens = lr_sens.predict_proba(X_sensitivity)[:, 1]
auroc_lr_sens = roc_auc_score(y, y_pred_lr_sens)
print(f"Sensitivity Logistic Regression AUROC: {auroc_lr_sens:.4f}")


In [ ]:
import xgboost as xgb

xgb_sens = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_sens.fit(X_sensitivity, y)

# Predict and compute AUROC
y_pred_xgb_sens = xgb_sens.predict_proba(X_sensitivity)[:, 1]
auroc_xgb_sens = roc_auc_score(y, y_pred_xgb_sens)
print(f"Sensitivity XGBoost AUROC: {auroc_xgb_sens:.4f}")


In [ ]:
import shap

explainer_sens = shap.Explainer(xgb_sens, X_sensitivity)
shap_values_sens = explainer_sens(X_sensitivity)

# Plot
shap.plots.bar(shap_values_sens, max_display=25)


In [ ]:
label_map = {
    'arf': 'Acute Respiratory Failure',
    'age': 'Age',
    'sepsis': 'Sepsis',
    'uti': 'Urinary Tract Infection',
    'aki': 'Acute Kidney Injury',
    'hosp_division': 'Hospital Division',
    'tran_in': 'Transferred from Another Facility',
    'asp': 'Aspiration Pneumonia',
    'elective': 'Elective Admission',
    'race': 'Race/Ethnicity',
    'zipinc_qrtl': 'Income Quartile (ZIP)',
    'chf': 'Congestive Heart Failure',
    'afib': 'Atrial Fibrillation',
    'anemia': 'Anemia',
    'malnut': 'Malnutrition',
    'dysph': 'Dysphagia',
    'pressulc': 'Pressure Ulcer',
    'cad': 'Coronary Artery Disease',
    'cva': 'Cerebrovascular Disease',
    'hypoth': 'Hypothyroidism',
    'dnr': 'Do-Not-Resuscitate',
    'pall': 'Palliative Care',
    'aweekend': 'Weekend Admission',
    'female': 'Female'
}

shap_df['Feature_Full'] = shap_df['Feature'].map(label_map)
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Sort by SHAP value (descending)
shap_df = shap_df.sort_values(by='SHAP_Mean_Abs', ascending=True)

# Plot
plt.figure(figsize=(12, 9))
plt.barh(shap_df['Feature_Full'], shap_df['SHAP_Mean_Abs'], color='deeppink')

# Larger font size for labels
plt.xlabel('mean(|SHAP value|)', fontsize=16)
plt.ylabel('Predictors', fontsize=16)
plt.title("Top Predictors of In-Hospital Mortality in AD (Sensitivity Analysis)", fontsize=18)

# Optionally increase tick label size
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.tight_layout()
plt.show()
